In [52]:
import pandas as pd
import numpy as np
import open3d as o3d
import time
import os
import json
import os
from data_repository.lookup_table import read_table
import time
import random
# import data_repository.lookup_table as lookup

In [53]:
def generate_unique_color(existing_colors):
    while True:
        color = list(random.randint(0, 255) for _ in range(3))
        if color not in existing_colors:
            return color

In [54]:
def get_all_data():
    objects = []
    look_up = read_table()
    path = 'data_repository/'

    ec = []
    
    for name in look_up[['objectID', 'instanceID']].values:
        ipath = path + f'{name[0]}.{name[1]}'
        
        # Check if the file exists and has content
        while not (os.path.isfile(ipath) and os.path.getsize(ipath) > 0):
            print(f"Waiting for the file {ipath} to be available and have content...")
            time.sleep(1)  # Wait for 1 second before checking again

        df = pd.read_csv(ipath)

        color = generate_unique_color(ec)
        ec.append(color)

        df[['r','g','b']] = [color[0]/255,color[1]/255,color[2]/255]
        
        objects.append(df)
    
    return pd.concat(objects, axis=0)

In [55]:
get_all_data()

,Unnamed: 0,X,Y,Z,r,g,b
0,0,3.697444,-31.386351,-0.883320,0.019608,0.219608,0.654902
1,1,3.693440,-31.368567,-0.870824,0.019608,0.219608,0.654902
2,2,3.696939,-31.369576,-0.853030,0.019608,0.219608,0.654902
3,3,3.704937,-31.404589,-0.876983,0.019608,0.219608,0.654902
4,4,3.708420,-31.406054,-0.860175,0.019608,0.219608,0.654902
...,...,...,...,...,...,...,...
5516,5516,2.177147,-33.358055,0.165487,0.698039,0.760784,0.866667
5517,5517,2.177146,-33.357388,0.180472,0.698039,0.760784,0.866667
5518,5518,2.177148,-33.358593,0.194213,0.698039,0.760784,0.866667
5519,5519,2.177148,-33.359833,0.207908,0.698039,0.760784,0.866667


In [56]:
def visualize_without_colors(df:pd.DataFrame, resx, resy, icm, ecm):
    """ 
    It opens x,y,z points using open3d

    parameters:
    - df:pd.DataFrame - it should only contain x, y, z columns
    """

    points = np.vstack([df['X'], df['Y'], df['Z']]).T

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(df[['r','g','b']].values)

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)
    camera_lines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=resx, view_height_px=resy, intrinsic=icm, extrinsic=ecm)
    vis.add_geometry(camera_lines)

    file_path = "data_repository/lookup_table.csv"

    last_modified_time = os.path.getmtime(file_path)
    print(time.ctime(last_modified_time))

    try:
        while True:
            current_modified_time = os.path.getmtime(file_path)
            if current_modified_time != last_modified_time:
                temp = get_all_data()
                points = np.vstack([temp['X'], temp['Y'], temp['Z']]).T

                if temp is not None:
                    pcd = o3d.geometry.PointCloud()
                    pcd.points = o3d.utility.Vector3dVector(points)
                    vis.clear_geometries()
                    vis.add_geometry(pcd)
                    vis.add_geometry(camera_lines)
                    
                last_modified_time = current_modified_time
            
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    except KeyboardInterrupt:
        print("Stopped by user")
        
    vis.destroy_window()

In [57]:
def get_loaded_camera_matrices(camera_path):
    with open(camera_path, 'r') as f:
        loaded_data = json.load(f)
        K_loaded = np.array(loaded_data['K'])
        RT_loaded = np.array(loaded_data['RT'])
        resx = int(loaded_data['resolution_x'])
        resy = int(loaded_data['resolution_y'])

    return resx, resy, K_loaded, RT_loaded

In [58]:
def is_valid_csv_file(file_path):
    """
    Checks if the given file path points to a valid CSV file.

    Args:
    - file_path (str): The file path to check.

    Returns:
    - bool: True if the file exists and has a '.csv' extension, False otherwise.
    """
    # Check if file exists
    if not os.path.isfile(file_path):
        return False

    # Check if file extension is '.csv'
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() != '.csv':
        return False

    return True


In [59]:
src_path = 'C:\\Programming\\Fourth Semester\\PointVIS\\PointVIS\\assets\\output\\'

id = 1

while True:
    camera_path = src_path + f'camera{id}.csv'
    if not is_valid_csv_file(camera_path):
        break
    id = id + 1

camera_path = src_path + f'camera{id - 1}.csv'

resx, resy, icm, ecm = get_loaded_camera_matrices(camera_path)
df = get_all_data()
visualize_without_colors(df, resx, resy, icm, ecm)

Wed Jun 19 20:09:42 2024
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: 